<a href="https://colab.research.google.com/github/dkshjn/Technoculture/blob/main/Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN_WRITE')
!huggingface-cli login --token $hf_token

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset

dataset = load_dataset("dkshjn/chatdoctor-200k-stripped-embedded")

Generating s1 split:   0%|          | 0/82964 [00:00<?, ? examples/s]

Generating s2 split:   0%|          | 0/82963 [00:00<?, ? examples/s]

Generating s3 split:   0%|          | 0/82963 [00:00<?, ? examples/s]

Generating s4 split:   0%|          | 0/82963 [00:00<?, ? examples/s]

Generating s5 split:   0%|          | 0/82963 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-small-en-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-small-en-v1.5')
model.eval()

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [5]:
def generate_embedding(input, output):
  embeddings_input = [input]
  embeddings_output = [output]
  encoded_input = tokenizer(embeddings_input, padding=True, truncation=True, return_tensors='pt')
  encoded_output = tokenizer(embeddings_output, padding=True, truncation=True, return_tensors='pt')
  # Compute token embeddings
  with torch.no_grad():
      # Input
      model_output_in = model(**encoded_input)
      sentence_embeddings_in = model_output_in[0][:, 0]

      # Output
      model_output_out = model(**encoded_output)
      sentence_embeddings_out = model_output_out[0][:, 0]

      return (sentence_embeddings_in, sentence_embeddings_out)

In [6]:
def insert_embeddings(row):
  embedding = generate_embedding(row["input"], row["output"])
  row["embeddings_input"] = embedding[0].numpy().tolist()[0]
  row["embeddings_output"] = embedding[1].numpy().tolist()[0]
  return row

In [9]:
new_dataset = dataset["s1"].select(list(range(10))).map(insert_embeddings)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [11]:
new_dataset

Dataset({
    features: ['input', 'instruction', 'output', 'embeddings_input', 'embeddings_output'],
    num_rows: 10
})

In [13]:
new_dataset.push_to_hub("dkshjn/chatdoctor-200k-stripped-embedded-v2", token=hf_token, split="s1")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/dkshjn/chatdoctor-200k-stripped-embedded-v2/commit/66083c9e75a9feb848b4729c67983bdc46453f3f', commit_message='Upload dataset', commit_description='', oid='66083c9e75a9feb848b4729c67983bdc46453f3f', pr_url=None, pr_revision=None, pr_num=None)